In [49]:
# Import Libraries	
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [50]:
# Loading Data
df = pd.read_csv('/Users/bee/Documents/Hacktiv8/Re-doing_project/GC4_/portugal_listinigs.csv')
df.head(5)

/var/folders/nk/tj46lgrj24l56dcl01d8smmw0000gn/T/ipykernel_42984/2256807460.py:2: DtypeWarning: Columns (6,8,12,13,14,15,16,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Users/bee/Documents/Hacktiv8/Re-doing_project/GC4_/portugal_listinigs.csv')


,Price,District,City,Town,Type,EnergyCertificate,Floor,Parking,HasParking,ConstructionYear,...,Elevator,ElectricCarsCharging,TotalRooms,NumberOfBedrooms,NumberOfWC,ConservationStatus,LivingArea,LotSize,BuiltArea,NumberOfBathrooms
0,250000.0,Faro,São Brás de Alportel,São Brás de Alportel,Apartment,A+,2nd Floor,1.0,True,NaN,...,True,NaN,2.0,NaN,NaN,NaN,114.0,NaN,NaN,2.0
1,9500.0,Faro,Albufeira,Albufeira e Olhos de Água,Apartment,NC,1st Floor,0.0,False,1990.0,...,True,NaN,0.0,NaN,NaN,NaN,27.0,NaN,NaN,1.0
2,580000.0,Faro,Vila do Bispo,Budens,Apartment,D,3rd Floor,1.0,True,2003.0,...,False,NaN,2.0,NaN,NaN,NaN,84.0,NaN,NaN,2.0
3,350000.0,Faro,Portimão,Portimão,Apartment,C,4th Floor,0.0,False,1985.0,...,True,NaN,2.0,NaN,NaN,NaN,68.0,NaN,NaN,1.0
4,175000.0,Faro,Faro,Faro (Sé e São Pedro),House,NC,NaN,0.0,False,1950.0,...,False,NaN,4.0,NaN,NaN,NaN,78.0,NaN,NaN,2.0


# Missing Values

In [51]:
# Cek Informasi Struktur Data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129837 entries, 0 to 129836
Data columns (total 25 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Price                  129541 non-null  float64
 1   District               129837 non-null  object 
 2   City                   129837 non-null  object 
 3   Town                   129835 non-null  object 
 4   Type                   129821 non-null  object 
 5   EnergyCertificate      129823 non-null  object 
 6   Floor                  27929 non-null   object 
 7   Parking                129643 non-null  float64
 8   HasParking             68215 non-null   object 
 9   ConstructionYear       84354 non-null   float64
 10  TotalArea              121922 non-null  float64
 11  GrossArea              27638 non-null   float64
 12  EnergyEfficiencyLevel  61590 non-null   object 
 13  PublishDate            25195 non-null   object 
 14  Garage                 61590 non-nul

In [52]:
# Cek Missing Value

# Total missing value per kolom
missing = df.isnull().sum()

# Persentase missing value
missing_percent = df.isnull().mean() * 100

# Gabungkan hasil
missing_data = pd.DataFrame({
    'Missing Values': missing,
    'Percent (%)': missing_percent
})

missing_data[missing_data['Missing Values'] > 0]

,Missing Values,Percent (%)
Price,296,0.227978
Town,2,0.001540
Type,16,0.012323
EnergyCertificate,14,0.010783
Floor,101908,78.489183
Parking,194,0.149418
HasParking,61622,47.461047
ConstructionYear,45483,35.030846
TotalArea,7915,6.096105
GrossArea,102199,78.713310


In [53]:
df['Floor'] = df['Floor'].astype(str).str.extract('(\d+)').astype(float)
df['Floor'] = df['Floor'].fillna(df['Floor'].median())

In [54]:
# Handliing Missiing Values

# Drop baris pada kolom dengan Missing Value Rendah (<1%)
df.dropna(subset=['Price', 'Town', 'Type', 'EnergyCertificate', 'Parking', 'Elevator'], inplace=True)

# ConstructionYear: isi dengan median per tipe properti (lebih representatif).
df['ConstructionYear'] = df.groupby('Type')['ConstructionYear'].transform(lambda x: x.fillna(x.median()))

# Floor: jika Type bukan apartemen/flat, bisa diisi 0 atau drop.
# df.loc[(df['Type'] != 'Apartment') & (df['Floor'].isna()), 'Floor'] = 0
# df['Floor_num'].fillna(df['Floor_num'].median(), inplace=True)

# Jika hanya satu jenis area yang tersedia, gunakan itu untuk mengisi yang lain
df['TotalArea'].fillna(df[['LivingArea', 'BuiltArea']].mean(axis=1), inplace=True)
df['LivingArea'].fillna(df['TotalArea'], inplace=True)
df['BuiltArea'].fillna(df['TotalArea'], inplace=True)

# GrossArea dan LotSize: gunakan median per tipe atau hapus jika terlalu banyak
df['GrossArea'].fillna(df.groupby('Type')['GrossArea'].transform('median'), inplace=True)
df['LotSize'].fillna(df.groupby('Type')['LotSize'].transform('median'), inplace=True)

# Gunakan median berdasarkan property Type:
for col in ['TotalRooms', 'NumberOfBedrooms', 'NumberOfWC', 'NumberOfBathrooms']:
    df[col].fillna(df.groupby('Type')[col].transform('median'), inplace=True)

# Isian boolean → isi False jika NaN (tidak punya fasilitas tsb)
for col in ['HasParking', 'Garage', 'ElectricCarsCharging']:
    df[col] = df[col].fillna(False)

# Diisi dengan mode berdasarkan EnergyCertificate jika tersedia
df['EnergyEfficiencyLevel'].fillna(df['EnergyEfficiencyLevel'].mode()[0], inplace=True)

# Jika tujuan analisis butuh waktu, drop. Jika tidak, abaikan
df.dropna(subset=['PublishDate'], inplace=True)

# Mengisi missing value kolom ConservationStatus dengan "Unknown"
df['ConservationStatus'] = df['ConservationStatus'].fillna('Unknown')


# Verifikasi hasil
df.isnull().sum()

/opt/miniconda3/lib/python3.9/site-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/var/folders/nk/tj46lgrj24l56dcl01d8smmw0000gn/T/ipykernel_42984/2941401377.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['TotalArea'].fillna(df[['LivingArea', 'BuiltArea']].mean(axis=1), inplace=True)
/var/folders/nk/tj46lgrj24l56dcl01d8smmw0000gn/T/ipykernel_42984/2941401377.py:15: FutureWarning: A value is trying to be set on a copy of a Data

Price                       0
District                    0
City                        0
Town                        0
Type                        0
EnergyCertificate           0
Floor                       0
Parking                     0
HasParking                  0
ConstructionYear            2
TotalArea                3096
GrossArea                   0
EnergyEfficiencyLevel       0
PublishDate                 0
Garage                      0
Elevator                    0
ElectricCarsCharging        0
TotalRooms                  0
NumberOfBedrooms            0
NumberOfWC                  0
ConservationStatus          0
LivingArea               3096
LotSize                     0
BuiltArea                3096
NumberOfBathrooms           0
dtype: int64

In [55]:
# Cek ulang missing value
df.isna().sum().sort_values(ascending=False)

BuiltArea                3096
LivingArea               3096
TotalArea                3096
ConstructionYear            2
Price                       0
Garage                      0
LotSize                     0
ConservationStatus          0
NumberOfWC                  0
NumberOfBedrooms            0
TotalRooms                  0
ElectricCarsCharging        0
Elevator                    0
EnergyEfficiencyLevel       0
PublishDate                 0
District                    0
GrossArea                   0
HasParking                  0
Parking                     0
Floor                       0
EnergyCertificate           0
Type                        0
Town                        0
City                        0
NumberOfBathrooms           0
dtype: int64

In [56]:
# ConstructionYear: tinggal isi median (karena hanya 2 missing)
df['ConstructionYear'].fillna(df['ConstructionYear'].median(), inplace=True)

# Untuk TotalArea, BuiltArea, LivingArea → saling isi jika satu ada, lainnya kosong
df['TotalArea'].fillna(df[['LivingArea', 'BuiltArea']].mean(axis=1), inplace=True)
df['BuiltArea'].fillna(df['TotalArea'], inplace=True)
df['LivingArea'].fillna(df['TotalArea'], inplace=True)

/var/folders/nk/tj46lgrj24l56dcl01d8smmw0000gn/T/ipykernel_42984/534176042.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['ConstructionYear'].fillna(df['ConstructionYear'].median(), inplace=True)
/var/folders/nk/tj46lgrj24l56dcl01d8smmw0000gn/T/ipykernel_42984/534176042.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which 

In [57]:
# Pastikan kita mengisi semua area menggunakan nilai yang tersedia dari kolom lain
df['TotalArea'].fillna(df[['LivingArea', 'BuiltArea']].mean(axis=1), inplace=True)
df['LivingArea'].fillna(df[['TotalArea', 'BuiltArea']].mean(axis=1), inplace=True)
df['BuiltArea'].fillna(df[['TotalArea', 'LivingArea']].mean(axis=1), inplace=True)

/var/folders/nk/tj46lgrj24l56dcl01d8smmw0000gn/T/ipykernel_42984/3555155226.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['TotalArea'].fillna(df[['LivingArea', 'BuiltArea']].mean(axis=1), inplace=True)
/var/folders/nk/tj46lgrj24l56dcl01d8smmw0000gn/T/ipykernel_42984/3555155226.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object o

In [58]:
df.isna().sum()

Price                       0
District                    0
City                        0
Town                        0
Type                        0
EnergyCertificate           0
Floor                       0
Parking                     0
HasParking                  0
ConstructionYear            0
TotalArea                3096
GrossArea                   0
EnergyEfficiencyLevel       0
PublishDate                 0
Garage                      0
Elevator                    0
ElectricCarsCharging        0
TotalRooms                  0
NumberOfBedrooms            0
NumberOfWC                  0
ConservationStatus          0
LivingArea               3096
LotSize                     0
BuiltArea                3096
NumberOfBathrooms           0
dtype: int64

In [59]:
area_median = df[['TotalArea', 'LivingArea', 'BuiltArea']].median()

df['TotalArea'].fillna(area_median['TotalArea'], inplace=True)
df['LivingArea'].fillna(area_median['LivingArea'], inplace=True)
df['BuiltArea'].fillna(area_median['BuiltArea'], inplace=True)

/var/folders/nk/tj46lgrj24l56dcl01d8smmw0000gn/T/ipykernel_42984/604606625.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['TotalArea'].fillna(area_median['TotalArea'], inplace=True)
/var/folders/nk/tj46lgrj24l56dcl01d8smmw0000gn/T/ipykernel_42984/604606625.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting

In [60]:
# Cek ulang missing value
df.isna().sum().sort_values(ascending=False)

Price                    0
PublishDate              0
BuiltArea                0
LotSize                  0
LivingArea               0
ConservationStatus       0
NumberOfWC               0
NumberOfBedrooms         0
TotalRooms               0
ElectricCarsCharging     0
Elevator                 0
Garage                   0
EnergyEfficiencyLevel    0
District                 0
GrossArea                0
TotalArea                0
ConstructionYear         0
HasParking               0
Parking                  0
Floor                    0
EnergyCertificate        0
Type                     0
Town                     0
City                     0
NumberOfBathrooms        0
dtype: int64

In [61]:
# Cek tipe data
df.dtypes


Price                    float64
District                  object
City                      object
Town                      object
Type                      object
EnergyCertificate         object
Floor                    float64
Parking                  float64
HasParking                  bool
ConstructionYear         float64
TotalArea                float64
GrossArea                float64
EnergyEfficiencyLevel     object
PublishDate               object
Garage                      bool
Elevator                  object
ElectricCarsCharging        bool
TotalRooms               float64
NumberOfBedrooms         float64
NumberOfWC               float64
ConservationStatus        object
LivingArea               float64
LotSize                  float64
BuiltArea                float64
NumberOfBathrooms        float64
dtype: object

# Inconsistent Data Types

In [62]:
# Handling Data Type

# Mengubah tipe data kolom Parking to int
df['Parking'] = pd.to_numeric(df['Parking'])

# Mengubah tipe data kolom HasParking to bool
df['HasParking'] = df['HasParking'].astype(bool)

# Mengubah tipe data kolom ConstructionYear to int
df['ConstructionYear'] = pd.to_numeric(df['ConstructionYear'])

# Mengubah tipe data kolom Garage to bool
df['Garage'] = df['Garage'].astype(bool)

# Mengubah tipe data kolom Elevator to bool
df['Elevator'] = df['Elevator'].astype(bool)

# Mengubah tipe data kolom ElectricCarsCharging to bool
df['ElectricCarsCharging'] = df['ElectricCarsCharging'].astype(bool)

# Mengubah tipe data kolom TotalRooms to int
df['TotalRooms'] = pd.to_numeric(df['TotalRooms'])

# Mengubah tipe data kolom NumberOfBedrooms to int
df['NumberOfBedrooms'] = pd.to_numeric(df['NumberOfBedrooms'])

# Mengubah tipe data kolom NumberOfWC to int
df['NumberOfWC'] = pd.to_numeric(df['NumberOfWC'])

# Mengubah tipe data kolom NumberOfBathrooms to int
df['NumberOfBathrooms'] = pd.to_numeric(df['NumberOfBathrooms'])


In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25182 entries, 68247 to 129833
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Price                  25182 non-null  float64
 1   District               25182 non-null  object 
 2   City                   25182 non-null  object 
 3   Town                   25182 non-null  object 
 4   Type                   25182 non-null  object 
 5   EnergyCertificate      25182 non-null  object 
 6   Floor                  25182 non-null  float64
 7   Parking                25182 non-null  float64
 8   HasParking             25182 non-null  bool   
 9   ConstructionYear       25182 non-null  float64
 10  TotalArea              25182 non-null  float64
 11  GrossArea              25182 non-null  float64
 12  EnergyEfficiencyLevel  25182 non-null  object 
 13  PublishDate            25182 non-null  object 
 14  Garage                 25182 non-null  bool   
 15  El

# Duplication

In [64]:
# Cek Duplikasi
# Jumlah data duplikat
df.duplicated().sum()

# Handling Duplikasi
df = df.drop_duplicates()

In [65]:
# Cek ulang duplikasi
df.duplicated().sum()

np.int64(0)

In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24192 entries, 68247 to 129833
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Price                  24192 non-null  float64
 1   District               24192 non-null  object 
 2   City                   24192 non-null  object 
 3   Town                   24192 non-null  object 
 4   Type                   24192 non-null  object 
 5   EnergyCertificate      24192 non-null  object 
 6   Floor                  24192 non-null  float64
 7   Parking                24192 non-null  float64
 8   HasParking             24192 non-null  bool   
 9   ConstructionYear       24192 non-null  float64
 10  TotalArea              24192 non-null  float64
 11  GrossArea              24192 non-null  float64
 12  EnergyEfficiencyLevel  24192 non-null  object 
 13  PublishDate            24192 non-null  object 
 14  Garage                 24192 non-null  bool   
 15  El

In [67]:
# Menyimpan data bersih to csv
df.to_csv('clean_portugal_listinigs.csv', index=False)
     